 - Para la creacion de un crud e interactuar con esto de la API REST de Django comenzamos con la creacion del modelo de "Product"

# 1. Modelo de Product

In [ ]:
from django.db import models

class Product(models.Model):
    name = models.CharField(max_length=255)
    description = models.TextField(blank=True, null=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    stock = models.PositiveIntegerField()

    def __str__(self):
        return self.name


# 2 Ejecutamos las migraciones despues de crearlo:

In [ ]:
python manage.py makemigrations
python manage.py migrate


# 3 Creamos un archivo serializador para convertir los objetos Python a JSON

In [ ]:
from rest_framework import serializers
from .models import Product

class ProductSerializer(serializers.ModelSerializer):
    class Meta:
        model = Product
        fields = '__all__'


# 4. Vistas con APIView

Implementamos los metodos:
- GET
- POST
- PUT
- DELETE

In [ ]:
from rest_framework import views, status
from rest_framework.response import Response
from .models import Product
from .serializers import ProductSerializer

class ProductAPIView(views.APIView):

    def get(self, request, pk=None):
        """Obtener lista de productos o un solo producto por ID"""
        if pk:
            try:
                product = Product.objects.get(pk=pk)
                serializer = ProductSerializer(product)
                return Response(serializer.data)
            except Product.DoesNotExist:
                return Response({"error": "Producto no encontrado"}, status=status.HTTP_404_NOT_FOUND)
        
        products = Product.objects.all()
        serializer = ProductSerializer(products, many=True)
        return Response(serializer.data)

    def post(self, request):
        """Crear un nuevo producto"""
        serializer = ProductSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    def put(self, request, pk):
        """Actualizar un producto por ID"""
        try:
            product = Product.objects.get(pk=pk)
        except Product.DoesNotExist:
            return Response({"error": "Producto no encontrado"}, status=status.HTTP_404_NOT_FOUND)
        
        serializer = ProductSerializer(product, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    def delete(self, request, pk):
        """Eliminar un producto por ID"""
        try:
            product = Product.objects.get(pk=pk)
        except Product.DoesNotExist:
            return Response({"error": "Producto no encontrado"}, status=status.HTTP_404_NOT_FOUND)

        product.delete()
        return Response({"message": "Producto eliminado"}, status=status.HTTP_204_NO_CONTENT)


# 5 Configuramos la rutas

In [ ]:
from django.urls import path
from .views import ProductAPIView

urlpatterns = [
    path("products/", ProductAPIView.as_view()),  # Lista y creación de productos
    path("products/<int:pk>/", ProductAPIView.as_view()),  # Detalle, actualización y eliminación de un producto
]


# 6 Pruebas

## Crear un post

### on the headers tab:

- key: Content-Type
- value: application/json

### Producto 1

URL: http://localhost:8000/api/v1/products/
BODY JSON:
{
    "name": "Laptop",
    "description": "Laptop Dell XPS 13",
    "price": 1200.50,
    "stock": 10
}


### Response

In [ ]:
{
    "id": 1,
    "name": "Laptop",
    "description": "Laptop Dell XPS 13",
    "price": "1200.50",
    "stock": 10
}

### Producto 2

URL: http://localhost:8000/api/v1/products/
BODY JSON:
{
    "name": "Gaming Chair",
    "description": "Raptor Gaming Chair",
    "price": 200.75,
    "stock": 7
}

### Response

In [ ]:
{
    "id": 2,
    "name": "Gaming Chair",
    "description": "Raptor Gaming Chair",
    "price": "200.75",
    "stock": 7
}

## Listar los productos

GET: http://localhost:8000/api/v1/products/

### Response

[
    {
        "id": 1,
        "name": "Laptop",
        "description": "Laptop Dell XPS 13",
        "price": "1200.50",
        "stock": 10
    },
    {
        "id": 2,
        "name": "Gaming Chair",
        "description": "Raptor Gaming Chair",
        "price": "200.75",
        "stock": 7
    }
]

## Obtener Producto por ID

GET:http://localhost:8000/api/v1/products/1

### Response

{
    "id": 1,
    "name": "Laptop",
    "description": "Laptop Dell XPS 13",
    "price": "1200.50",
    "stock": 10
}

## Actualizar producto por ID

PUT: http://localhost:8000/api/v1/products/1/

Body:
    {
    "name": "Laptop Gaming",
    "description": "Laptop Alienware",
    "price": 2500.99,
    "stock": 5
}


### Response

{
    "id": 1,
    "name": "Laptop Gaming",
    "description": "Laptop Alienware",
    "price": "2500.99",
    "stock": 5
}

## Elliminar Producto por ID

DELETE: http://localhost:8000/api/v1/products/1/

### Response

{
    "message": "Producto eliminado"
}